In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from num2words import num2words
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

# ------------------------------
# 1. Train MNIST digit recognizer
# ------------------------------
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(10, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=3, batch_size=64, validation_data=(x_test, y_test))

# ------------------------------
# 2. Helper functions
# ------------------------------
def segment_digits(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (400, 100))  # resize for consistency
    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    digit_imgs = []

    for cnt in sorted(contours, key=lambda x: cv2.boundingRect(x)[0]):  # left to right
        x, y, w, h = cv2.boundingRect(cnt)
        if h > 30:  # filter noise
            digit = thresh[y:y+h, x:x+w]
            digit = cv2.resize(digit, (28, 28))
            digit = digit.reshape(28, 28, 1).astype("float32") / 255.0
            digit_imgs.append(digit)

    return np.array(digit_imgs)

def predict_number(image_path):
    digits = segment_digits(image_path)
    preds = model.predict(digits)
    numbers = [np.argmax(p) for p in preds]
    full_number = int("".join(map(str, numbers)))
    print("Predicted Number:", full_number)
    print("In Words:", num2words(full_number))

    plt.imshow(cv2.imread(image_path))
    plt.title(f"Predicted: {full_number}")
    plt.axis("off")
    plt.show()

# ------------------------------
# 3. Interactive Upload Loop (fixed)
# ------------------------------
stop_button = widgets.Button(description="Stop", button_style="danger")
upload_button = widgets.Button(description="Upload Image", button_style="info")
output = widgets.Output()
stopped = {"flag": False}

def stop_clicked(b):
    stopped["flag"] = True
    with output:
        clear_output()
        print("🔴 Stopped by user.")

def upload_clicked(b):
    if stopped["flag"]:
        with output:
            clear_output()
            print("⚠️ Upload stopped. Restart the notebook to continue.")
        return
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f"Processing {filename}...")
        predict_number(filename)

stop_button.on_click(stop_clicked)
upload_button.on_click(upload_clicked)

display(upload_button, stop_button, output)

